In [ ]:
################################################ VARIANTES GERMINALES ######################################################
################################################  HAPLOTYPECALLER ######################################################

## LLAMADO DE VARIANTES
reference="/home/yordonez3/Danny_OV_WES/Concatenated_fastq/GRCh38_Gencode31/hg38.fa"
output_dir="/home/yordonez3/Danny_OV_WES/Concatenated_fastq/EXOMA/variantes"
dbsnp_file="/home/yordonez3/Danny_OV_WES/GATK_recursos/Homo_sapiens_assembly38.dbsnp138.vcf"
num_threads=14

mkdir -p "$output_dir/gvcfs"

for bam_file in "$output_dir"/*-BQSR.bam; do
  sample_name=$(basename "$bam_file" -BQSR.bam)

  gatk --java-options "-Xmx35g" HaplotypeCaller \
    -R "$reference" \
    -I "$bam_file" \
    -O "$output_dir/gvcfs/${sample_name}.g.vcf.gz" \
    -ERC GVCF \
    --dbsnp "$dbsnp_file" \
    --native-pair-hmm-threads "$num_threads"
    
  echo "HaplotypeCaller completado para $sample_name."
done

In [ ]:
## GENOMIC IMPORT

reference=reference="/home/yordonez3/Danny_OV_WES/concatenated_fastq/concatenated_all_fastq/hg38.fa"
output_dir="reference="/home/yordonez3/Danny_OV_WES/concatenated_fastq/VARIANTES/gvcfs"

inputs=""
for file in $output_dir/gvcfs/*.g.vcf.gz; do
  inputs+=" -V $file"
done

gatk --java-options "-Xmx4g -Xms4g" GenomicsDBImport \
  $inputs \
  --genomicsdb-workspace-path my_database \
  --tmp-dir $output_dir/gvcfs/ \
  --intervals chr1 \
  --intervals chr2 \
  --intervals chr3 \
  --intervals chr4 \
  --intervals chr5 \
  --intervals chr6 \
  --intervals chr7 \
  --intervals chr8 \
  --intervals chr9 \
  --intervals chr10 \
  --intervals chr11 \
  --intervals chr12 \
  --intervals chr13 \
  --intervals chr14 \
  --intervals chr15 \
  --intervals chr16 \
  --intervals chr17 \
  --intervals chr18 \
  --intervals chr19 \
  --intervals chr20 \
  --intervals chr21 \
  --intervals chr22 \
  --intervals chrX \
  --intervals chrY
    
echo "GenomicsDBImport completado."


In [ ]:
## GENOTIPIFICACION

reference="/home/yordonez3/Danny_OV_WES/concatenated_fastq/concatenated_all_fastq/hg38.fa"
gatk GenotypeGVCFs \
    -R "$reference" \
    -V gendb://my_database \
    -O genotipos_exo_27.vcf

# Ver el archivo VCF y contar variantes
bcftools view genotipos_chr1.vcf.gz | head -n 20
bcftools view -H -v snps genotipos_chr1.vcf.gz | wc -l


In [ ]:
## 10. FILTRADO DE VARIANTES

reference="/home/yordonez3/Danny_OV_WES/concatenated_fastq/concatenated_all_fastq/hg38.fa"

gatk VariantFiltration \
     -R "$reference" \
     -V genotipos_exo_27.vcf  \
     --window 35 \
     --cluster 3 \
     --filter-name "FS" \
     --filter "FS > 30.0" \
     --filter-name "QD" \
     --filter "QD < 2.0" \
     --filter-name "MQ" \
     --filter "MQ < 30.0" \
     -O geno_exo_57_filtered.vcf.g


## cerrando el pc
#### reference="/home/wovalle3/Samples/GRCh38_Gencode31/hg38.fa"

reference="/home/yordonez3/Danny_OV_WES/concatenated_fastq/concatenated_all_fastq/hg38.fa"

#### nohup gatk VariantFiltration \

gatk VariantFiltration \
     -R "$reference" \
     -V genotipos_exo_27.vcf  \
     --window 35 \
     --cluster 3 \
     --filter-name "FS" \
     --filter "FS > 30.0" \
     --filter-name "QD" \
     --filter "QD < 2.0" \
     --filter-name "MQ" \
     --filter "MQ < 30.0" \
     -O geno_exo_27_filtered.vcf.g

   ####  > variantfiltration.log 2>&1 &

# Contar los SNPs obtenidos
bcftools view -H -v snps geno_exo_27_filtered.vcf.gz | wc -l
# 8.999.354
# 26.335.989
# 30.338.751

#Criterios para el filtrado
FS (Phred-scaled p-value using Fisher's exact test to detect strand bias)
FS > 30.0: Filters variants with a Fisher Strand (FS) score greater than 30.0.

QD es el valor de calidad de una variante dividido por la profundidad de cobertura en ese sitio específico
QD < 2.0: Filters variants with a Quality by Depth (QD) score less than 2.0.
variantes con un valor QD menor que 2.0 serán marcadas como filtradas.
    
MQ Mapping Quality MQ ≥ 30, es un umbral comúnmente utilizado en muchos estudios. 
Se considera que un MQ de 30 o más indica una alineación confiable.

In [ ]:
###PLINK

mv geno_exo_27_filtered.vcf.gz geno_exo_27_filtered.vcf.gz.tbi /home/yordonez3/Danny_OV_WES/concatenated_fastq/EXOMA/variantes/plink

#Antes de empezar revisar que el archivo tenga los IDs de los SNPs
bcftools view --no-header geno_exo_27_filtered.vcf.gz | head


In [ ]:
###PLINK
VCF de entrada: geno_exo_57_filtered.vcf.gz

bcftools view --no-header geno_exo_57_filtered.vcf.gz | head

##1. Generar los bfiles con el archivo .vcf de mis muestras
The first step for quality control (QC) analysis in PLINK starting from
your VCF files is to convert the VCF file to PLINK format (BED, BIM, FAM).
PLINK provides a tool called plink2 for this purpose. 
                                 
plink --vcf geno_exo_57_filtered.vcf.gz --make-bed --out geno_plink

#Results 28 samples
Total genotyping rate is 0.461463.
14.853.128 variants and 18 people pass filters and QC.

Total genotyping rate is 0.434183.
20.581.021 variants and 28 people pass filters and QC.


#results 46 samples
Total genotyping rate is 0.397116.
28.508.607 variants and 46 people pass filters and QC.
Note: No phenotypes present.
    
#results 57 samples
Total genotyping rate is 0.38405.
32.742.195 variants and 57 people pass filters and QC

In [ ]:
#2. Perform QC analysis

#Opciones de QC
#Removing individuals with high missing genotype rates (--mind): --mind 0.05
#Removing SNPs with high missingness (--geno): --geno 0.05
#Removing SNPs with low minor allele frequency (--maf): --maf 0.01
#Performing Hardy-Weinberg equilibrium (HWE) testing (--hwe): 1e-6
#Removing SNPs with extreme heterozygosity (--het).
#Identificación de SNPs Duplicados (--list-duplicate-vars)
#Filtrar por Identidad por Descendencia (IBD) (--genome y --rel-cutoff)

#Usar comando geno 0.1 para que deje solamente SNPs con CR>90%
plink --bfile geno_plink --geno 0.1  --make-bed --out geno_plink_fil1

#Results
Total genotyping rate is 0.461463.
12.763.929 variants removed due to missing genotype data (--geno).
2.089.199 variants and 18 people pass filters and QC.

Total genotyping rate is 0.434183.
18.721.953 variants removed due to missing genotype data (--geno).
1.859.068 variants and 28 people pass filters and QC.

#results 46
Total genotyping rate is 0.397116.
25.802.351 variants removed due to missing genotype data (--geno).
2.706.256 variants and 46 people pass filters and QC.
Note: No phenotypes present.
    
#57
Total genotyping rate is 0.38405.
29.499.078 variants removed due to missing genotype data (--geno).
3.243.117 variants and 57 people pass filters and QC.

#Control de calidad para individuos con mind. Deja solamente casos CR>90%
plink --bfile geno_plink_fil1 --mind 0.25 --make-bed --out geno_plink_fil2   ##Mejor hacerlo con 0.1 despues

#Results
Total genotyping rate in remaining samples is 0.943978.
2.089.199 variants and 17 people pass filters and QC.

Total genotyping rate is 0.97497.
1.859.068 variants and 28 people pass filters and QC.

#results 46
Total genotyping rate is 0.972913.
2.706.256 variants and 46 people pass filters and QC.

#57
Total genotyping rate is 0.973838.
3243117 variants and 57 people pass filters and QC.

In [ ]:
#3. Filtrar por MAF y HW

#MAF: 0.01 remueve variantes con frecuencia de alelo menor a 1% (es dcir mutaciones y demas)
plink --bfile geno_plink_fil2 --maf 0.01 --make-bed --out geno_plink_fil3

#Results
Total genotyping rate is 0.943978
124.280 variants removed due to minor allele threshold(s)
1.964.919 variants and 17 people pass filters and QC.

Total genotyping rate is 0.97497.
3041 variants removed due to minor allele threshold(s)
1.856.027 variants and 28 people pass filters and QC

#results 46
Total genotyping rate is 0.972913.
2042 variants removed due to minor allele threshold(s)
(--maf/--max-maf/--mac/--max-mac).
2.704.214 variants and 46 people pass filters and QC.

#57
Total genotyping rate is 0.973838.
2188097 variants removed due to minor allele threshold(s)
(--maf/--max-maf/--mac/--max-mac).
1.055.020 variants and 57 people pass filters and QC.


#HW
plink --bfile geno_plink_fil3 --hwe 1e-6 --hwe-all --make-bed --out geno_plink_fil4

#Results
Total genotyping rate is 0.944006.
--hwe: 960 variants removed due to Hardy-Weinberg exact test.
1.963.959 variants and 17 people pass filters and QC.

Total genotyping rate is 0.974968.
--hwe: 520 variants removed due to Hardy-Weinberg exact test.
1.855.507 variants and 28 people pass filters and QC.

#results 46
Total genotyping rate is 0.972912.
--hwe: 1333 variants removed due to Hardy-Weinberg exact test.
2.702.881 variants and 46 people pass filters and QC.

#57
Total genotyping rate is 0.968644.
--hwe: 2072 variants removed due to Hardy-Weinberg exact test.
1.052.948 variants and 57 people pass filters and QC.

In [ ]:
# Perform LD pruning in two steps:
# Step 1: Generate list of SNPs to keep
# We perform linkage disequilibrium (LD) pruning using:
  # - a window size of 50 SNPs
  # - a step size of 5 SNPs
  # - r2 threshold of 0.2 (correlation between SNPs)

plink --bfile geno_plink_fil4 --indep-pairwise 50 5 0.2 --out ld_pruned

#Results 
Total genotyping rate is 0.944005.
1.963.959 variants and 17 people pass filters and QC.
Pruning complete.  1.668.064 of 1.963.959 variants removed.

#results 46
Pruning complete.  1848160 of 2702881 variants removed.


# Step 2: Apply LD pruning
plink --bfile geno_plink_fil4 --extract ld_pruned.prune.in --make-bed --out geno_plink_fil5

#Results 
Ambiguous sex IDs written to geno_plink_fil5.nosex .
--extract: 1963959 variants remaining.
Warning: At least 295894 duplicate IDs in --extract file.
Total genotyping rate is 0.944005.
1.963.959 variants and 17 people pass filters and QC.

Total genotyping rate is 0.974963.
1.855.507 variants and 28 people pass filters and QC

#results 46
Total genotyping rate is 0.972908.
2.702.881 variants and 46 people pass filters and QC.


In [ ]:
#4. Set-missing-var-ids para mis datos y el panel de referencia

#El comando set-missing-var-ids en PLINK se utiliza para asignar 
#identificadores (IDs) a los SNPs que no tienen un ID en el archivo BIM

plink --bfile geno_plink_fil5 --set-missing-var-ids chr@:#:\$2:\$1 --make-bed --out genoexo_NoMissVar
    
#Results
Total genotyping rate is 0.944005.
1.963.959 variants and 17 people pass filters and QC.

Total genotyping rate is 0.974963.
1.855.507 variants and 28 people pass filters and QC.

#results 46
Total genotyping rate is 0.972908.
2.702.881 variants and 46 people pass filters and QC.

#57
Total genotyping rate is 0.968623.
1.052.948 variants and 57 people pass filters and QC.


#Esto lo hago desde la carpeta de FinalReferencePanel si no se tiene 
#plink --bfile extractedChrAll --set-missing-var-ids chr@:#:\$2:\$1 --make-bed --out extractedChrAll_NoMissVar
    
#Results
23.886.383 variants loaded from .bim file.
969 people (0 males, 0 females, 969 ambiguous) loaded from .fam.
Total genotyping rate is 0.999872.
23.886.383 variants and 969 people pass filters and QC.

#Mover los archivos a la carpeta desde donde se esta haciendo el analisis
mv extractedChrAll_NoMissVar* /home/wovalle3/Samples/total_samples_pool/EXOMA/variantes/plink

In [ ]:
#5. Encontrar variantes que se comparten entre la base de referencia y la mia

#Para asegurarte de que el panel de referencia tiene exactamente las mismas 
#variantes que tu archivo antes de realizar un merge en PLINKls

comm -12 <(sort genoexo_NoMissVar.bim) <(sort extractedChrAll_NoMissVar.bim) > variantes_comunes.txt
wc -l variantes_comunes.txt #119.776 compartidas  #111.293   #116.467 #97.632

plink --bfile extractedChrAll_NoMissVar --extract variantes_comunes.txt --make-bed --out Ref_comunes

#Results
Total genotyping rate is 0.99987.
119.776 variants and 969 people pass filters and QC

Total genotyping rate is 0.999874.
111.293 variants and 969 people pass filters and QC.

#results 46
Total genotyping rate is 0.999877.
116.467 variants and 969 people pass filters and QC.

#57
Total genotyping rate is 0.999876.
97.632 variants and 969 people pass filters and QC.



plink --bfile genoexo_NoMissVar --extract variantes_comunes.txt --make-bed --out genoexo_comunes
#Results
Total genotyping rate is 0.95625.
119.776 variants and 17 people pass filters and QC.

Total genotyping rate is 0.978565.
111.293 variants and 28 people pass filters and QC.

#results 46
Total genotyping rate is 0.975627.
116.467 variants and 46 people pass filters and QC.

#57
Total genotyping rate is 0.975575.
97.632 variants and 57 people pass filters and QC.

In [ ]:
#6. Merge del panel de referencia con mis datos

plink --bfile Ref_comunes --bmerge genoexo_comunes --make-bed --out merged_set_28

(969 + 17)
Total genotyping rate is 0.999118.
119.776 variants and 986 people pass filters and QC.

Total genotyping rate is 0.999275.
111.293 variants and 997 people pass filters and QC.

#results 46
Total genotyping rate is 0.998778.
116.467 variants and 1015 people pass filters and QC.

#57
Total genotyping rate is 0.998526.
97.632 variants and 1026 people pass filters and QC.


plink --bfile merged_set_28 --geno 0.10  --make-bed --out merged_set_fil28

#results
0 variants removed due to missing genotype data (--geno).
119776 variants and 986 people pass filters and QC.

0 variants removed due to missing genotype data (--geno).
111293 variants and 997 people pass filters and QC

0 variants removed due to missing genotype data (--geno).
116467 variants and 1015 people pass filters and QC.

0 variants removed due to missing genotype data (--geno).
97632 variants and 1026 people pass filters and QC


#PCA con los resultados
plink --bfile merged_set_fil28 --pca --out pca_results_57
#mac
scp -r wovalle3@bio-compjord01.biosci.gatech.edu:/home/wovalle3/Samples/total_samples_pool/EXOMA/variantes/plink/pca_results_57.eigenval /Users/wen/Documents/
scp -r wovalle3@bio-compjord01.biosci.gatech.edu:/home/wovalle3/Samples/total_samples_pool/EXOMA/variantes/plink/pca_results_57.eigenvec /Users/wen/Documents/

#pc INC
scp -r wovalle3@bio-compjord01.biosci.gatech.edu:/home/wovalle3/Samples/total_samples_pool/EXOMA/variantes/plink/pca_results_28.eigenval C:\Users\wmontero\Documents
scp -r wovalle3@bio-compjord01.biosci.gatech.edu:/home/wovalle3/Samples/total_samples_pool/EXOMA/variantes/plink/pca_results_28.eigenvec C:\Users\wmontero\Documents
#Los archivos generados por PLINK para el análisis de PCA (pca_results.eigenvec y pca_results.eigenval) 
#son archivos de texto que contienen los resultados del análisis de componentes principales. 
#Puedes abrir y analizar estos archivos en R para visualización y análisis adicionales

In [ ]:
#9.	Generar un nuevo archivo recodificado para Admixture

plink --bfile merged_set_fil28 --recode12 --out Final_admixture57

#results
Total genotyping rate is 0.999118.
119.776 variants and 986 people pass filters and QC.

Total genotyping rate is 0.999275.
111.293 variants and 997 people pass filters and QC.

#results 46
Total genotyping rate is 0.998778.
116.467 variants and 1015 people pass filters and QC.

#57
Total genotyping rate is 0.998526.
97.632 variants and 1026 people pass filters and QC.

#Instalar Admixture en el servidor 
wget https://dalexander.github.io/admixture/binaries/admixture_linux-1.3.0.tar.gz

#Descomprimir el archivo tar.gz
tar -zxvf admixture_linux-1.3.0.tar.gz

In [ ]:
#10. Correr admixture desde donde esta el archivo ped
/home/wovalle3/ADMIXTURE/dist/admixture_linux-1.3.0/admixture Final_admixture57.ped 3

#Results
Summary: 
Converged in 13 iterations (239.995 sec)
Loglikelihood: -85308375.284505
Fst divergences between estimated populations: 
	Pop0	Pop1	
Pop0	
Pop1	0.136	
Pop2	0.181	0.123	
Writing output files.

Converged in 16 iterations (249.68 sec)
Loglikelihood: -71583396.008181
Fst divergences between estimated populations: 
	Pop0	Pop1	
Pop0	
Pop1	0.134	
Pop2	0.182	0.126	
Writing output files.

#results 46


#Pasar archivo Final_admixture.3.Q y merged_set.fam a pc para mirar resultados
#mac
scp -r wovalle3@bio-compjord01.biosci.gatech.edu:/home/wovalle3/Samples/total_samples_pool/EXOMA/variantes/plink/Final_admixture57.3.Q /Users/wen/Documents/
scp -r wovalle3@bio-compjord01.biosci.gatech.edu:/home/wovalle3/Samples/total_samples_pool/EXOMA/variantes/plink/merged_set_28.fam /Users/wen/Documents/

#pc INC
scp -r wovalle3@bio-compjord01.biosci.gatech.edu:/home/wovalle3/Samples/total_samples_pool/EXOMA/variantes/plink/Final_admixture28.3.Q C:\Users\wmontero\Documents
scp -r wovalle3@bio-compjord01.biosci.gatech.edu:/home/wovalle3/Samples/total_samples_pool/EXOMA/variantes/plink/merged_set_28.fam C:\Users\wmontero\Documents